# Проект: Анализ взаимодействия пользователей с карточками статей Яндекс. Дзен

Перед командой аналитиков Дзена поставлена задача по автоматизации анализа пользовательского взаимодействия с карточками статей Яндекс.Дзен и построении дашборда для менеджеров по анализу контента.

**Техническое задание:**

Бизнес-задача: анализ взаимодействия пользователей с карточками Дзена;

Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;

Кто будет основным пользователем дашборда: менеджеры по анализу контента;

**Состав данных для дашборда:**

История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);

Разбивка событий по темам источников;

Таблица соответствия тем источников темам карточек;

**По каким параметрам данные должны группироваться:**

Дата и время;

Тема карточки;

Тема источника;

Возрастная группа;

**Характер данных:**

История событий по темам карточек — абсолютные величины с разбивкой по минутам;

Разбивка событий по темам источников — относительные величины (% событий);

Соответствия тем источников темам карточек - абсолютные величины;

Важность: все графики имеют равную важность;


### Описание данных

Источники данных для дашборда: агрегирующая таблица **dash_visits**.
**Вот её структура:**

record_id — первичный ключ,

item_topic — тема карточки,

source_topic — тема источника,

age_segment — возрастной сегмент,

dt — дата и время,

visits — количество событий.

Таблица хранится в специально подготовленной базе данных zen;

Частота обновления данных: один раз в сутки, в полночь по UTC;

## Загрузка данных

Создадим коннекцию к базе.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': '***', # имя пользователя
            'pwd': '***', # пароль
            'host': '***',
            'port': ***, # порт подключения
            'db': '***'} # название базы данных
            
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
# формируем sql-запрос.
query='''SELECT * FROM dash_visits'''

In [3]:
# выполняем запрос и сохраняем результат
dash_visits = pd.io.sql.read_sql(query, con = engine)

In [4]:
# выведем на экран первые 5 строк датасета
print(dash_visits.head(5))

   record_id item_topic source_topic age_segment                  dt  visits
0    1040597     Деньги         Авто       18-25 2019-09-24 18:32:00       3
1    1040598     Деньги         Авто       18-25 2019-09-24 18:35:00       1
2    1040599     Деньги         Авто       18-25 2019-09-24 18:54:00       4
3    1040600     Деньги         Авто       18-25 2019-09-24 18:55:00      17
4    1040601     Деньги         Авто       18-25 2019-09-24 18:56:00      27


In [5]:
# посмотрим за какой период содержит данные датафрейм
# найдем минимальную дату
date_min = dash_visits['dt'].min()
# найдем максимальную дату
date_max = dash_visits['dt'].max()
print('Выгрузка содержит данные за период с ',date_min,'по',date_max)

Выгрузка содержит данные за период с  2019-09-24 18:28:00 по 2019-09-24 19:00:00


In [6]:
# проверим есть ли строки с нулевым количеством визитов
dash_visits[dash_visits['visits'] == 0]

,record_id,item_topic,source_topic,age_segment,dt,visits


In [7]:
# выгрузим данные из таблицы dash_visits в файл dash_visits.csv
dash_visits.to_csv('dash_visits.csv',index=False)
dash_visits.to_excel('dash_visits2.xlsx',index=False)